In [1]:
import os
import sys
from glob import glob
import random
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from tqdm import tqdm

import torch
import torch.nn as nn
from torchvision import transforms
import torchvision.transforms.functional as F

import cv2
import monai
from monai.losses import DiceLoss, DiceCELoss
from monai.metrics import DiceMetric
from monai.inferers import sliding_window_inference
from monai.data import CacheDataset, DataLoader, Dataset, decollate_batch
from monai.transforms import (
    AsDiscrete,
    AsDiscreted,
    EnsureChannelFirstd,
    Compose,
    CropForegroundd,
    RandFlipd,
    RandShiftIntensityd,
    RandRotate90d,
    LoadImaged,
    Orientationd,
    RandCropByPosNegLabeld,
    SaveImaged,
    ScaleIntensityRanged,
    Spacingd,
    Invertd,
)
import SimpleITK as sitk
from scipy.spatial.distance import directed_hausdorff

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html


In [2]:
min_intensity = -175
max_intensity = 250

In [3]:
# Loading training set
random.seed(53)

images = glob('DLP_dataset/training/images' + "/*.mha", recursive=True)
masks = glob('DLP_dataset/training/masks' + "/*.mha", recursive=True)

images = sorted(images)
masks = sorted(masks)

data_list = [{"img": img, "mask": mask} for img, mask in zip(images, masks)]

num_train = int(0.8*len(data_list))

num_val = len(data_list)-num_train

train_list = random.sample(data_list, num_train)

for element in train_list:
    data_list.remove(element)

val_list = data_list

print("There are: {} train and {} validation samples.".format(len(train_list), len(val_list)))

There are: 40 train and 10 validation samples.


In [4]:
print(train_list)

[{'img': 'DLP_dataset/training/images/subject042_CTA.mha', 'mask': 'DLP_dataset/training/masks/subject042_label.mha'}, {'img': 'DLP_dataset/training/images/subject015_CTA.mha', 'mask': 'DLP_dataset/training/masks/subject015_label.mha'}, {'img': 'DLP_dataset/training/images/subject031_CTA.mha', 'mask': 'DLP_dataset/training/masks/subject031_label.mha'}, {'img': 'DLP_dataset/training/images/subject034_CTA.mha', 'mask': 'DLP_dataset/training/masks/subject034_label.mha'}, {'img': 'DLP_dataset/training/images/subject049_CTA.mha', 'mask': 'DLP_dataset/training/masks/subject049_label.mha'}, {'img': 'DLP_dataset/training/images/subject032_CTA.mha', 'mask': 'DLP_dataset/training/masks/subject032_label.mha'}, {'img': 'DLP_dataset/training/images/subject035_CTA.mha', 'mask': 'DLP_dataset/training/masks/subject035_label.mha'}, {'img': 'DLP_dataset/training/images/subject025_CTA.mha', 'mask': 'DLP_dataset/training/masks/subject025_label.mha'}, {'img': 'DLP_dataset/training/images/subject051_CTA.mha

In [5]:
print(val_list)

[{'img': 'DLP_dataset/training/images/subject001_CTA.mha', 'mask': 'DLP_dataset/training/masks/subject001_label.mha'}, {'img': 'DLP_dataset/training/images/subject006_CTA.mha', 'mask': 'DLP_dataset/training/masks/subject006_label.mha'}, {'img': 'DLP_dataset/training/images/subject010_CTA.mha', 'mask': 'DLP_dataset/training/masks/subject010_label.mha'}, {'img': 'DLP_dataset/training/images/subject017_CTA.mha', 'mask': 'DLP_dataset/training/masks/subject017_label.mha'}, {'img': 'DLP_dataset/training/images/subject026_CTA.mha', 'mask': 'DLP_dataset/training/masks/subject026_label.mha'}, {'img': 'DLP_dataset/training/images/subject029_CTA.mha', 'mask': 'DLP_dataset/training/masks/subject029_label.mha'}, {'img': 'DLP_dataset/training/images/subject045_CTA.mha', 'mask': 'DLP_dataset/training/masks/subject045_label.mha'}, {'img': 'DLP_dataset/training/images/subject050_CTA.mha', 'mask': 'DLP_dataset/training/masks/subject050_label.mha'}, {'img': 'DLP_dataset/training/images/subject052_CTA.mha

In [6]:

# Transform
train_transforms = Compose(
    [
        LoadImaged(keys=["img", "mask"], ensure_channel_first=True),
        ScaleIntensityRanged(
            keys=["img"],
            a_min=min_intensity,
            a_max=max_intensity,
            b_min=0.0,
            b_max=1.0,
            clip=True,
        ),
        CropForegroundd(keys=["img", "mask"], source_key="img"),
        Orientationd(keys=["img", "mask"], axcodes="RAS"),
        Spacingd(keys=["img", "mask"], pixdim=(2, 2, 2), mode=("bilinear", "nearest")),
        RandCropByPosNegLabeld(
            keys=["img", "mask"],
            label_key="mask",
            spatial_size=(128,128,128),
            pos=1,
            neg=1,
            num_samples=4,
            image_key="img",
            image_threshold=0,
        ),
        RandFlipd(keys=["img", "mask"],spatial_axis=[0],prob=0.10),
        RandFlipd(keys=["img", "mask"],spatial_axis=[1],prob=0.10),
        RandFlipd(keys=["img", "mask"],spatial_axis=[2],prob=0.10),
        RandRotate90d(keys=["img", "mask"],prob=0.10,max_k=3)
    ]
)
val_transforms = Compose(
    [
        LoadImaged(keys=["img", "mask"], ensure_channel_first=True),
        ScaleIntensityRanged(
            keys=["img"],
            a_min=min_intensity,
            a_max=max_intensity,
            b_min=0.0,
            b_max=1.0,
            clip=True,
        ),
        CropForegroundd(keys=["img", "mask"], source_key="img"),
        Orientationd(keys=["img", "mask"], axcodes="RAS"),
        Spacingd(keys=["img", "mask"], pixdim=(2, 2, 2), mode=("bilinear", "nearest"))

    ]
)

monai.transforms.croppad.dictionary CropForegroundd.__init__:allow_smaller: Current default value of argument `allow_smaller=True` has been deprecated since version 1.2. It will be changed to `allow_smaller=False` in version 1.5.


In [7]:
num_cpus = torch.get_num_threads()

In [8]:
train_ds = CacheDataset(train_list, train_transforms, cache_num=len(train_list), cache_rate=1, num_workers = num_cpus)
train_loader = DataLoader(train_ds, batch_size=1, shuffle=True, num_workers=num_cpus, pin_memory=True)

Loading dataset: 100%|██████████| 40/40 [03:11<00:00,  4.78s/it]


In [9]:
val_ds = CacheDataset(val_list, val_transforms, cache_num=len(val_list), cache_rate=1, num_workers = num_cpus)
val_loader = DataLoader(val_ds, batch_size=1, num_workers=num_cpus, pin_memory=True)

Loading dataset: 100%|██████████| 10/10 [00:45<00:00,  4.59s/it]


In [10]:
num_classes = 24

In [11]:
swin_unetr = monai.networks.nets.SwinUNETR(
    img_size=128,
    spatial_dims=3,
    in_channels=1,
    out_channels=num_classes
).cuda()


loss_function = DiceCELoss(to_onehot_y=True, softmax=True)
optimizer = torch.optim.Adam(swin_unetr.parameters(), lr=1e-4)
dice_metric = DiceMetric(include_background=False, reduction="mean", get_not_nans=False, num_classes=num_classes)

monai.networks.nets.swin_unetr SwinUNETR.__init__:img_size: Argument `img_size` has been deprecated since version 1.3. It will be removed in version 1.5. The img_size argument is not required anymore and checks on the input size are run during forward().


### SWIN-UNETR Training

In [12]:
# start training
total_epoch = 500
best_dice = 0.0

for epoch in range(total_epoch):
    swin_unetr.train()
    epoch_loss = 0.0
    num_images = 0
    for batch in tqdm(train_loader):
        images = batch['img'].cuda()
        masks = batch['mask'].cuda()
        num_images += 4

        optimizer.zero_grad()
        outputs = swin_unetr(images)

        loss = loss_function(outputs, masks)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

    epoch_loss = epoch_loss/num_images
    print('Epoch {} loss: {}'.format(epoch, epoch_loss))
    
    if (epoch + 1) % 5 == 0:
        swin_unetr.eval()
        print('Begin to validate!!')
        with torch.no_grad():
            val_idx=0
            val_loss=0
            for batch in tqdm(val_loader):
                val_idx += 1
                val_img = batch['img'].cuda()
                val_mask = batch['mask'].cuda()

                
                roi_size = (128,128,128)
                sw_batch_size = 4
                val_output = sliding_window_inference(val_img, roi_size, sw_batch_size, swin_unetr)
                
                loss = loss_function(val_output, val_mask)
                
                val_loss += loss.item()            
    
                val_output = torch.argmax(val_output, dim=1, keepdim=True)
                dice_metric(y_pred=val_output, y=val_mask)
                    
            # aggregate the final mean dice result
            mean_dice = dice_metric.aggregate().item()
            # reset the status for next validation round
            dice_metric.reset()
                
            val_loss = val_loss/val_idx
            
            print('Validation ended !!')
            print('Mean Dice: {}, Loss: {}'.format(mean_dice, val_loss))

            if mean_dice > best_dice:
                best_dice = mean_dice
                torch.save(swin_unetr.state_dict(), 'SWIN_UNETR_model.pth')
            print('End of Validation-----------------------')
            
print('Training ended !!')  

100%|██████████| 40/40 [01:14<00:00,  1.85s/it]


Epoch 0 loss: 0.9985556781291962


100%|██████████| 40/40 [00:37<00:00,  1.05it/s]


Epoch 1 loss: 0.897250910103321


100%|██████████| 40/40 [00:38<00:00,  1.05it/s]


...

Epoch 498 loss: 0.1627186167985201


100%|██████████| 40/40 [00:37<00:00,  1.05it/s]


Epoch 499 loss: 0.1594765905290842
Begin to validate!!


100%|██████████| 10/10 [00:16<00:00,  1.60s/it]

Validation ended !!
Mean Dice: 0.6504436731338501, Loss: 0.35427051186561587
End of Validation-----------------------
Training ended !!
